# Flows.jl

In [1]:
using Flows

INFO: Recompiling stale cache file /home/hofi/.julia/lib/v0.4/Flows.ji for module Flows.


## TimeExpression

### TimeVariable

In [2]:
t = TimeVariable("t")

t

In [3]:
typeof(t)

Flows.TimeVariable

In [4]:
@t_vars s r τ

(s,r,τ)

In [5]:
typeof(s)

Flows.TimeVariable

### TimeLinearCombination

In [78]:
ex = TimeLinearCombination(t,1,s,-2,r,3)

t-2s+3r

The internal representation of a `TimeLinearCombination` is a list of (`TimeExpression`, coefficient)-pairs:

In [79]:
ex.terms

3-element Array{Tuple{Flows.TimeExpression,Real},1}:
 (t,1) 
 (s,-2)
 (r,3) 

The operators +, -, * are overloaded. Therefore, the `TimeLinearCombination` `ex` from above can also be defined in the following way:

In [80]:
ex1 = t - 2s + 3r

t-2s+3r

In [81]:
ex == ex1

true

In [82]:
string(ex)

"t-2s+3r"

In [83]:
t-2ex+11r-2(r-s)

-t+6s+3r

In [84]:
ex-ex

t_zero

In [85]:
string(ex-ex)

"t_zero"

### t_zero
There is a predefined zero element `t_zero` which is implemented as an empty `TimeLinearCombination`:

In [12]:
ex-ex == t_zero

true

In [13]:
typeof(t_zero)

Flows.TimeLinearCombination

In [14]:
t_zero.terms

0-element Array{Tuple{Flows.TimeExpression,Real},1}

### coefficient

In [15]:
ex

t-2s+3r

In [16]:
coefficient(ex,s)

-2

### substitute

In [17]:
substitute(ex,s,ex)

-t+4s-3r

In [18]:
substitute(t,s,r)

t

## SpaceExpression
Here 'space' is ot necessarily physical space, but abstract (Banach) space on which functions (i.e. operators) and flows of differential equations are defined.

### SpaceVariable

In [19]:
u = SpaceVariable("u")

u

In [20]:
@x_vars v,w

(v,w)

### AutonomousFunction

In [21]:
F = AutonomousFunction("F")

F

In [22]:
@funs G,H

(G,H)

In [23]:
typeof(G)

Flows.AutonomousFunction

### AutonomousFunctionExpression

In [24]:
ex = AutonomousFunctionExpression(F,u)

F(u)

In [25]:
ex1 = F(u)

F(u)

In [26]:
string(ex1)

"F(u)"

In [27]:
ex == ex1

true

`AutonomousFunctionExpression` involving differentials:

In [28]:
ex = F(u,v)

F(u,v)

In [29]:
string(ex)

"F(u,v)"

In [30]:
ex = F(u,v,w)

F(u,v,w)

In [31]:
string(ex)

"F(u,v,w)"

### FlowExpression

In [32]:
ex = FlowExpression(F, t, u, 0) # (the last argument 0 indicates that it is a 0th derivative w.r.t. time)

E(F,t,u)

In [33]:
ex1 = E(F,t,u)

E(F,t,u)

In [34]:
ex == ex1

true

In [35]:
string(ex)

"E(F,t,u)"

`FlowExpression` involving differentials:

In [36]:
ex = E(F,t,u,v)

E(F,t,u,v)

In [37]:
string(ex)

"E(F,t,u,v)"

In [38]:
ex = E(F,t,u,v,w)

E(F,t,u,v,w)

In [39]:
string(ex)

"E(F,t,u,v,w)"

### NonAutonomousFunction

In [40]:
@nonautonomous_funs(S)

(S,)

### NonAutonomousFunctionExpression

In [41]:
ex = S(t,u)

S(t,u)

In [42]:
string(ex)

"S(t,u)"

In [43]:
ex = S(2,t,u,v)

S(2,t,u,v)

In [44]:
string(ex)

"S(2,t,u,v)"

### SpaceLinearCombinations

In [86]:
ex = -17E(F,t,u,v,w) + 2u + F(v,w)

F(v,w)+2u-17E(F,t,u,v,w)

In [87]:
ex-ex

x_zero

In [88]:
string(ex)

"F(v,w)+2u-17E(F,t,u,v,w)"

### x_zero

In [89]:
ex-ex == x_zero

true

In [90]:
string(ex-ex)

"x_zero"

### differential

In [50]:
ex = F(G(u)) + E(F,t,u,w)

F(G(u))+E(F,t,u,w)

In [51]:
string(ex)

"F(G(u))+E(F,t,u,w)"

We form the differential of the expression `ex` with respect to the variable `u` and apply it to the expression `H(v)`. Note that the differntial is a linear map which has to be applied to something ("the slots have to be filled").

In [52]:
differential(ex, u, H(v))

E(F,t,u,w,H(v))+F(G(u),G(u,H(v)))

In [53]:
ex = S(1,t-2s,2u+v)

S(1,t-2s,2u+v)

In [54]:
expand(differential(ex, u, w))

2S(1,t-2s,2u+v,w)

### t_derivative

In [55]:
ex = E(F,t-2s,u+E(G,s,v))

E(F,t-2s,u+E(G,s,v))

In [56]:
t_derivative(ex, s)

-2F(E(F,t-2s,u+E(G,s,v)))+E(F,t-2s,u+E(G,s,v),G(E(G,s,v)))

In [57]:
ex = S(2t,S(t,v))

S(2t,S(t,v))

In [58]:
t_derivative(ex, t)

S(2t,S(t,v),S(1,t,v))+2S(1,2t,S(t,v))

### expand

In [59]:
ex = E(F,t,u,2v+3w)

E(F,t,u,2v+3w)

In [60]:
Flows.expand(ex)

2E(F,t,u,v)+3E(F,t,u,w)

In [61]:
ex = G(u, v+w, v+w)

G(u,v+w,v+w)

In [62]:
Flows.expand(ex)

G(u,v,v)+G(u,w,w)+2G(u,v,w)

In [63]:
ex = S(t,u,2v+3w)

S(t,u,2v+3w)

In [64]:
expand(ex)

2S(t,u,v)+3S(t,u,w)

### reduce_order

In [65]:
ex = E(F,t,u,F(u))

E(F,t,u,F(u))

In [66]:
reduce_order(ex)

F(E(F,t,u))

In [67]:
ex = E(F,t,u,F(u),v)

E(F,t,u,F(u),v)

In [68]:
reduce_order(ex)

-E(F,t,u,F(u,v))+F(E(F,t,u),E(F,t,u,v))

In [69]:
ex = E(F,t,u,F(u),v, w)

E(F,t,u,F(u),v,w)

In [70]:
reduce_order(ex)

-E(F,t,u,F(u,v),w)+F(E(F,t,u),E(F,t,u,v,w))-E(F,t,u,F(u,v,w))+F(E(F,t,u),E(F,t,u,v),E(F,t,u,w))-E(F,t,u,F(u,w),v)

### substitute

In [71]:
substitute(E(G,t,v), v, E(F,t,u))

E(G,t,E(F,t,u))

Commutator $[F,G]$:

In [72]:
C_FG = F(u,G(u))-G(u,F(u))

F(u,G(u))-G(u,F(u))

Double commutator $[F,[F,G]]$ by substituting $G\to[F,G]$ in $[F,G]$:

In [73]:
expand(substitute(C_FG, G, C_FG, u))

-F(u,G(u),F(u))-2F(u,G(u,F(u)))+G(u,F(u),F(u))+F(u,F(u,G(u)))+G(u,F(u,F(u)))

### commutator

In [74]:
commutator(F,G,u)

F(u,G(u))-G(u,F(u))

In [75]:
commutator(F,G,H,u)

H(u,G(u),F(u))+H(u,G(u,F(u)))-G(u,H(u,F(u)))-G(u,H(u),F(u))+F(u,-H(u,G(u))+G(u,H(u)))

Verify Jacobi identity $[F,[G,H]]+[H,[F,G]]+[G,[H,F]]=0$:

In [76]:
expand(commutator(F,G,H,u)+commutator(G,H,F,u)+commutator(H,F,G,u))

x_zero